In [2]:
#일반 라이브러리
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#API를 사용하기 위한 import
# from geoband.API import *
# import folium
# import json
# import geopandas as gpd

#model
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression


In [3]:
police9 = pd.read_csv("police9.csv")
police9 = police9.iloc[:,1:]
police9.head()

,jur_stn,year,상반기,파출소수,경찰관수,5대범죄발생건,5대범죄검거건,5대범죄검거율,전체인구,전체성비,...,중학교,고등학교,"대학교(2,3년제)",대학교(4년제 이상),대학원(석박사 과정),받지 않았음(미취학 포함),1인당 담당인구수,기초생활자수,지방세징수액,1인가구비율
0,수원남부경찰서,2017,0,9,395,3184,1417,0.445038,405665,102.4,...,0.086358,0.309598,0.128393,0.285944,0.056419,0.022521,1027,0.085953,598246,0.093576
1,수원남부경찰서,2017,1,9,395,2967,1426,0.480620,405665,102.4,...,0.086358,0.309598,0.128393,0.285944,0.056419,0.022521,1027,0.028378,598246,0.093576
2,수원남부경찰서,2018,0,9,395,3143,1378,0.438435,406281,102.3,...,0.086358,0.309598,0.128393,0.285944,0.056419,0.022521,1028,0.090206,568278,0.095834
3,수원남부경찰서,2018,1,9,395,2825,1374,0.486372,406281,102.3,...,0.086358,0.309598,0.128393,0.285944,0.056419,0.022521,1028,0.086160,568278,0.095834
4,수원남부경찰서,2019,0,9,395,2589,1725,0.666280,404995,102.3,...,0.086358,0.309598,0.128393,0.285944,0.056419,0.022521,1025,0.103774,509559,0.100109


In [4]:
df = pd.read_csv("5.112신고.csv")
df.date = df.date.astype("str")
df["year"] = df.date.str[:4]
temp = df.date.str[4:6] <= "06"
df["상반기"] = temp.astype("int")
df = df.drop("date", axis = 1)
df.head()

FileNotFoundError: [Errno 2] File 5.112신고.csv does not exist: '5.112신고.csv'

In [ ]:
def count_case(crimes):
    df_violence = df[df["inc_info"].isin(crimes)]
    sgg_list = df_violence["jur_stn"].unique().tolist()
    ans = pd.DataFrame(index = range(0,0), columns= ["year","상반기","jur_stn","city"])

    for sgg in sgg_list:
        temp = df_violence[df_violence["jur_stn"] == sgg]
        df1 = temp.groupby(["year","상반기"])["jur_stn"].count().reset_index()
        df1["city"] = sgg
        ans = pd.concat([ans,df1])

    # 이름바꾸기    
    ans.rename(columns = {"jur_stn": "cnt"}, inplace = True)
    return ans

def print_case():
    case = df["inc_info"].unique()
    case = case.tolist()
    for idx, crim in enumerate(case):
        print(idx+1,crim, end = " ")
        if (idx+1) % 7 == 0:
            print("\n")
            
# 원하는사건/전체사건 비율 구하기
def make_proportion(df1):
    city_list = df1["city"].unique().tolist()
    years = ["2018","2019","2020","2021"]
    half = [0,1]

    for city in city_list:
        for year in years:
            for quarter in half:

                try: 
                    value = total_112[(city,year,quarter)]
                    df2 = df1[(df1["city"] == city) & (df1["상반기"] == quarter) & (df1["year"] == year)]
                    proportion = df2["cnt"] / value
                    df1.loc[(df1["city"] == city) & (df1["상반기"] == quarter) & (df1["year"] == year), "cnt"] = proportion
                    #print(df1[(df1["city"] == city) & (df1["year"] == year)])

                except:
                    pass
                    #print("Fail: {}, {}".format(city,year))
    df1 = df1.astype({"cnt":"float"})
    return df1

#18,19년으로 바꾸기 + city이름 바꾸기
def chagne2police(df):
    df = df[df["year"].isin(["2018","2019"])]
    df.city = df.city.str[:] + "경찰서"
    df = df.rename(columns = {"city":"jur_stn"})
    df = df.astype({"year":"int",
                      "상반기":"int"})
    return df

In [ ]:
# 전체 사건개수
total_112 = dict(df.groupby(["jur_stn","year","상반기"])["inc_info"].count())

# 절도폭력
df1 = count_case(["절도","폭력","학교폭력","성폭력","가정폭력","데이트폭력","아동학대(가정내)","아동학대(기타)"])
df1 = make_proportion(df1)
df1 = chagne2police(df1)

# 강도살인
# 강제적인 느낌을 담기면 모두 넣자
df2 = count_case(["강도","살인","납치감금","변사자"])
df2 = make_proportion(df2)
df2 = chagne2police(df2)

# 교통사고
df3 = count_case(["교통사고","교통위반","교통불편","인피도주"])
df3 = make_proportion(df3)
df3 = chagne2police(df3)

# 기초질서/집회시위질서
# 도박추가, 기타경범 추가, 주거침입
df4 = count_case(["소음","행패소란","위험방지","주취자","도박","재물손괴","청소년비행","시비","기타경범","무전취식승차","가출 등","수배불심자","풍속영업","주거침입","스토킹","노점상","위험동물"])
df4 = make_proportion(df4)
df4 = chagne2police(df4)

In [ ]:
df1 = df1.rename(columns = {"cnt":"절폭신고비율"})
df2 = df2.rename(columns = {"cnt":"강살신고비율"})
df3 = df3.rename(columns = {"cnt":"교통신고비율"})
df4 = df4.rename(columns = {"cnt":"질서신고비율"})
column_list = ["절폭신고비율", "강살신고비율", "교통신고비율", "질서신고비율"]
df4.head()

In [ ]:
police10 = pd.merge(police9, df1, how = "outer", on = ["jur_stn","year","상반기"])
police10 = pd.merge(police10, df2, how = "outer", on = ["jur_stn","year","상반기"])
police10 = pd.merge(police10, df3, how = "outer", on = ["jur_stn","year","상반기"])
police10 = pd.merge(police10, df4, how = "outer", on = ["jur_stn","year","상반기"])

In [ ]:
dataset = police10.drop(["jur_stn","year","상반기"], axis = 1)
dataset.head()

In [ ]:
def find_nan(column):
    train = dataset[dataset[column].isnull() == False] 
    test = dataset[dataset[column].isnull() == True]
    X_train, Y_train = train.drop(column_list, axis = 1), train[column]
    X_test = test.drop(column_list,axis = 1)

    model = LinearRegression(normalize= True)
    model.fit(X_train,Y_train)
    pred = model.predict(X_test)
    police10.loc[police10[column].isnull(),column] = pred


for column in column_list:
    find_nan(column)


In [ ]:
#police10.to_csv("police10.csv")